In [15]:
# The '%' tells Jupyter to install it in the specific environment the notebook is using
%pip install pandas
%pip install neo4j


Note: you may need to restart the kernel to use updated packages.
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [neo4j]━━━━━ 1/2 [neo4j]
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import re
import os

# This lists everything in your data folder so we can see the exact names
data_path = "../data/resume/"
if os.path.exists(data_path):
    print("Contents of ../data/:")
    print(os.listdir(data_path))
else:
    print("The path '../data/' was not found. Are you sure you are in the 'pipelines' folder?")

Contents of ../data/:
['UpdatedResumeDataSet.csv']


In [13]:


# --- PATH CONFIGURATION ---
# Based on your directory structure: intelligent-recruiter/pipelines/ -> ../data/
BASE_DATA_PATH = "../data/skill_data/"
RESUME_PATH = "../data/resume/UpdatedResumeDataSet.csv" # Confirm your filename in /resume
OUTPUT_PATH = "../output/"

# Ensure output directory exists
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

def run_pipeline():
    print("🚀 Loading ESCO and Resume data...")
    
    # 1. Load Files using your specific folder names
    skills = pd.read_csv(f"{BASE_DATA_PATH}skills_en.csv")
    occs = pd.read_csv(f"{BASE_DATA_PATH}occupations_en.csv")
    rels = pd.read_csv(f"{BASE_DATA_PATH}occupationSkillRelations_en.csv")
    resumes = pd.read_csv(RESUME_PATH)
    
    # 2. Build Skill Dictionary for matching
    # ESCO provides unique URIs for every skill
    skill_list = skills['preferredLabel'].dropna().unique().tolist()
    
    print(f"🔍 Matching {len(resumes)} resumes against {len(skill_list)} ESCO skills...")

    # Accurate matching with word boundaries to avoid partial matches
    def extract_skills(text):
        text = str(text).lower()
        found = []
        for skill in skill_list:
            skill_clean = skill.lower()
            if re.search(r'\b' + re.escape(skill_clean) + r'\b', text):
                found.append(skill_clean)
        return list(set(found))

    # Apply extraction
    resumes['matched_skills'] = resumes['Resume'].apply(extract_skills)

    # 3. Export CSVs for Neo4j Ingestion
    print(f"📄 Exporting files to {OUTPUT_PATH}...")
    
    # Nodes
    skills[['conceptUri', 'preferredLabel']].to_csv(f"{OUTPUT_PATH}nodes_skills.csv", index=False)
    occs[['conceptUri', 'preferredLabel']].to_csv(f"{OUTPUT_PATH}nodes_occupations.csv", index=False)
    resumes[['Category']].to_csv(f"{OUTPUT_PATH}nodes_candidates.csv", index_label='candidate_id')

    # Edges: Occupation -> REQUIRES -> Skill
    # relationType identifies if a skill is essential or optional
    rels[['occupationUri', 'skillUri', 'relationType']].to_csv(f"{OUTPUT_PATH}edges_occ_skill.csv", index=False)

    # Edges: Candidate -> HAS_SKILL -> Skill
    skill_to_uri = dict(zip(skills['preferredLabel'].str.lower(), skills['conceptUri']))
    candidate_skill_edges = []
    for idx, row in resumes.iterrows():
        for s_name in row['matched_skills']:
            uri = skill_to_uri.get(s_name)
            if uri:
                candidate_skill_edges.append({'candidate_id': idx, 'skillUri': uri})
    
    pd.DataFrame(candidate_skill_edges).to_csv(f"{OUTPUT_PATH}edges_candidate_skill.csv", index=False)
    print("✅ Done! Files ready in /output folder.")

run_pipeline()

🚀 Loading ESCO and Resume data...
🔍 Matching 962 resumes against 13939 ESCO skills...
📄 Exporting files to ../output/...
✅ Done! Files ready in /output folder.


In [1]:
from neo4j import GraphDatabase
import pandas as pd

# Connection details
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "12345678"

def load_data_to_neo4j():
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    with driver.session() as session:
        print("🧹 Cleaning existing data...")
        session.run("MATCH (n) DETACH DELETE n") # Optional: Clears graph before loading

        # 1. Load Skills
        print("📥 Loading Skill Nodes...")
        skills_df = pd.read_csv("../output/nodes_skills.csv")
        session.run("""
            UNWIND $data AS row
            MERGE (s:Skill {uri: row.conceptUri})
            SET s.name = row.preferredLabel
        """, data=skills_df.to_dict('records'))

        # 2. Load Occupations
        print("📥 Loading Occupation Nodes...")
        occs_df = pd.read_csv("../output/nodes_occupations.csv")
        session.run("""
            UNWIND $data AS row
            MERGE (o:Occupation {uri: row.conceptUri})
            SET o.name = row.preferredLabel
        """, data=occs_df.to_dict('records'))

        # 3. Load Candidates
        print("📥 Loading Candidate Nodes...")
        cands_df = pd.read_csv("../output/nodes_candidates.csv")
        session.run("""
            UNWIND $data AS row
            MERGE (c:Candidate {id: row.candidate_id})
            SET c.category = row.Category
        """, data=cands_df.to_dict('records'))

        # 4. Load Candidate -> Skill Relationships
        print("🔗 Linking Candidates to Skills...")
        cand_skill_df = pd.read_csv("../output/edges_candidate_skill.csv")
        session.run("""
            UNWIND $data AS row
            MATCH (c:Candidate {id: row.candidate_id})
            MATCH (s:Skill {uri: row.skillUri})
            MERGE (c)-[:HAS_SKILL]->(s)
        """, data=cand_skill_df.to_dict('records'))

        # 5. Load Occupation -> Skill Relationships (The ESCO Taxonomy)
        print("🔗 Linking Occupations to Skills (Taxonomy)...")
        occ_skill_df = pd.read_csv("../output/edges_occ_skill.csv")
        session.run("""
            UNWIND $data AS row
            MATCH (o:Occupation {uri: row.occupationUri})
            MATCH (s:Skill {uri: row.skillUri})
            MERGE (o)-[r:REQUIRES]->(s)
            SET r.type = row.relationType
        """, data=occ_skill_df.to_dict('records'))

    driver.close()
    print("✅ Neo4j Ingestion Complete!")

load_data_to_neo4j()

🧹 Cleaning existing data...
📥 Loading Skill Nodes...
📥 Loading Occupation Nodes...
📥 Loading Candidate Nodes...
🔗 Linking Candidates to Skills...
🔗 Linking Occupations to Skills (Taxonomy)...
✅ Neo4j Ingestion Complete!
